#### Importing libraries

In [ ]:
import numpy as np 
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import cv2

In [ ]:
from cycler import cycler
import matplotlib as mpl

raw_light_palette = [
    (0, 122, 255), # Blue
    (255, 149, 0), # Orange
    (52, 199, 89), # Green
    (255, 59, 48), # Red
    (175, 82, 222),# Purple
    (255, 45, 85), # Pink
    (88, 86, 214), # Indigo
    (90, 200, 250),# Teal
    (255, 204, 0)  # Yellow
]
raw_dark_palette = [
    (10, 132, 255), # Blue
    (255, 159, 10), # Orange
    (48, 209, 88),  # Green
    (255, 69, 58),  # Red
    (191, 90, 242), # Purple
    (94, 92, 230),  # Indigo
    (255, 55, 95),  # Pink
    (100, 210, 255),# Teal
    (255, 214, 10)  # Yellow
]
raw_gray_light_palette = [
    (142, 142, 147),# Gray
    (174, 174, 178),# Gray (2)
    (199, 199, 204),# Gray (3)
    (209, 209, 214),# Gray (4)
    (229, 229, 234),# Gray (5)
    (242, 242, 247),# Gray (6)
]
raw_gray_dark_palette = [
    (142, 142, 147),# Gray
    (99, 99, 102),  # Gray (2)
    (72, 72, 74),   # Gray (3)
    (58, 58, 60),   # Gray (4)
    (44, 44, 46),   # Gray (5)
    (28, 28, 39),   # Gray (6)
]

light_palette = np.array(raw_light_palette)/255
dark_palette = np.array(raw_dark_palette)/255
gray_light_palette = np.array(raw_gray_light_palette)/255
gray_dark_palette = np.array(raw_gray_dark_palette)/255

mpl.rcParams['axes.prop_cycle'] = cycler('color',dark_palette)
mpl.rcParams['figure.facecolor']  = gray_dark_palette[-2]
mpl.rcParams['figure.edgecolor']  = gray_dark_palette[-2]
mpl.rcParams['axes.facecolor'] =  gray_dark_palette[-2]

white_color = gray_light_palette[-2]
mpl.rcParams['text.color'] = white_color
mpl.rcParams['axes.labelcolor'] = white_color
mpl.rcParams['axes.edgecolor'] = white_color
mpl.rcParams['xtick.color'] = white_color
mpl.rcParams['ytick.color'] = white_color

mpl.rcParams['figure.dpi'] = 200
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False

In [ ]:
# Create a list with the filepaths for training and testing
train_img_Path = '../input/plant-pathology-2021-fgvc8/train_images'

test_img_Path = '../input/plant-pathology-2021-fgvc8/test_images'

img_Path = '../input/resized-plant2021/img_sz_256'

train = pd.read_csv(r'../input/plant-pathology-2021-fgvc8/train.csv')

sample_submission = pd.read_csv(r'../input/plant-pathology-2021-fgvc8/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
print(f'Number of pictures in the training dataset: {train.shape[0]}\n')
print(f'Number of different labels: {len(train.labels.unique())}\n')
print(f'Labels: {train.labels.unique()}')

In [ ]:
train['labels'].value_counts()

In [ ]:
CLASSES = train['labels'].unique().tolist()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Preprocessing the Training set
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.1,
                                   zoom_range = 0.1,
                                   horizontal_flip = True,
                                   validation_split=0.25)

train_data = train_datagen.flow_from_dataframe(train,
                                              directory=img_Path,
                                              classes=CLASSES,
                                              x_col="image",
                                              y_col="labels",
                                              target_size=(150, 150),
                                              subset='training')

val_data = train_datagen.flow_from_dataframe(train,
                                            directory=img_Path,
                                            classes=CLASSES,
                                            x_col="image",
                                            y_col="labels",
                                            target_size=(150, 150),
                                            subset='validation')

In [ ]:
dict_classes = train_data.class_indices
dict_classes

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from tensorflow.keras.layers import Conv2D, Dropout,MaxPooling2D,Flatten,Dense
from tensorflow.keras.applications.resnet_v2 import ResNet50V2

In [ ]:
base_resnet50V2 = ResNet50V2(include_top = False, 
                         weights = '../input/keras-pretrained-models/ResNet50V2_NoTop_ImageNet.h5', 
                         input_shape = train_data.image_shape, 
                         pooling='avg',
                         classes = CLASSES)

In [ ]:
#Adding the final layers to the above base models where the actual classification is done in the dense layers
model_ResNet = Sequential()
model_ResNet.add(base_resnet50V2)
model_ResNet.add(Dense(12, activation=('softmax')))

model_ResNet.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_ResNet.summary()

# Training the CNN on the Train data and evaluating it on the val data
r = model_ResNet.fit(train_data, validation_data = val_data, epochs = 15, batch_size=32)

In [ ]:
model_ResNet.save('res101ver1.h5')

In [ ]:
# test_dir = '/kaggle/input/plant-pathology-2021-fgvc8/test_images/'
# test_df = pd.DataFrame()
# test_df['image'] = os.listdir(test_dir)

# test_data = train_datagen.flow_from_dataframe(dataframe=test_df,
#                                     directory=test_dir,
#                                     x_col="image",
#                                     y_col=None,
#                                     batch_size=32,
#                                     seed=42,
#                                     shuffle=False,
#                                     class_mode=None,
#                                     target_size=(150, 150))

In [ ]:
# pred1 = tf.keras.models.load_model('../input/incepres/Iresnet50ver2.h5')
# pred2 = tf.keras.models.load_model('../input/res101/res101ver1.h5')


In [ ]:
# pred1 = pred1.predict(test_data)
# pred2 = pred2.predict(test_data)


In [ ]:
# pred = (pred1+pred2).tolist()

In [ ]:
# for i in range(len(pred)):
#     pred[i] = np.argmax(pred[i])

    
# def get_key(val):
#     for key, value in dict_classes.items():
#         if val == value:
#             return key
        

# for i in range(len(pred)):
#     pred[i] = get_key(pred[i])

In [ ]:
# print(pred)

In [ ]:
# test_df['labels'] = pred
# # test_df

In [ ]:
# test_df.to_csv('submission.csv',index=False)